In [1]:
import pandas as pd

In [ ]:
capacidades = pd.read_csv('data/Capacidades.csv')


In [17]:
horarios = pd.read_csv('data/HorariosMaterias.csv')
horarios = horarios.melt(id_vars=['Materia-Curso'], value_name='Bloque').drop(columns=['variable']).dropna()
horarios

,Materia-Curso,Bloque
0,T7-A,M2
1,T7-B,J3
2,T7-C,M3
3,T7-D,V3
4,T7-E,V1
...,...,...
85,T2-O,J4
86,T2-P,J2
87,T2-Q,X4
88,T2-R,X2


In [ ]:
class Profe:
    def __init__(self, id : int, bloques_asignados : int, disponibilidad : list):
        self.id = id
        self.bloques_asignados = bloques_asignados
        self.disponibilidad = disponibilidad

In [19]:
disponibilidades = pd.read_csv('data/Disponibilidad.csv')
disponibilidades.drop(columns=['Unnamed: 0'], inplace=True)
disponibilidades.melt(id_vars=['PROF #', 'Bloques asignados'], var_name = 'Bloque', value_name='Disponibilidad')

,PROF #,Bloques asignados,Bloque,Disponibilidad
0,PROF 1,12,L1,x
1,PROF 2,10,L1,x
2,PROF 3,9,L1,x
3,PROF 4,10,L1,x
4,PROF 5,2,L1,x
...,...,...,...,...
385,PROF 9,2,V6,NaN
386,PROF 10,2,V6,NaN
387,PROF 11,4,V6,NaN
388,PROF 12,4,V6,NaN


In [ ]:

capacidades.drop(columns=['Unnamed: 0'], inplace=True)


,Unnamed: 0,PROF #,Bloques asignados,L1,L2,L3,L4,L5,L6,M1,...,J3,J4,J5,J6,V1,V2,V3,V4,V5,V6
0,ALTIERI,PROF 1,12,x,NaN,NaN,NaN,NaN,NaN,x,...,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BOSSIO,PROF 2,10,x,x,x,x,x,NaN,x,...,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CLERICI,PROF 3,9,x,x,x,x,x,NaN,x,...,x,x,x,NaN,x,x,x,x,x,NaN
3,DI_FONZO,PROF 4,10,x,x,x,x,x,NaN,x,...,x,x,x,NaN,x,x,x,x,NaN,NaN
4,FLAMINIO,PROF 5,2,x,x,x,NaN,NaN,NaN,x,...,x,x,NaN,NaN,x,x,x,NaN,NaN,NaN
5,FRYDMAN,PROF 6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GARBER,PROF 7,6,x,x,x,NaN,NaN,NaN,x,...,x,NaN,NaN,NaN,x,x,x,NaN,NaN,NaN
7,MARTINEZ,PROF 8,10,x,x,x,x,x,NaN,x,...,x,x,x,NaN,x,x,x,x,x,NaN
8,MASCIANICA,PROF 9,2,NaN,NaN,NaN,NaN,NaN,NaN,x,...,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PETELIN,PROF 10,2,NaN,NaN,NaN,NaN,NaN,NaN,x,...,x,x,x,NaN,x,x,x,x,x,NaN


,Unnamed: 0,PROF #,T7,T72,T1,T2
0,ALTIERI,PROF 1,NaN,NaN,NaN,x
1,BOSSIO,PROF 2,NaN,NaN,x,NaN
2,CLERICI,PROF 3,NaN,x,x,x
3,DI_FONZO,PROF 4,NaN,NaN,x,NaN
4,FLAMINIO,PROF 5,NaN,x,NaN,NaN
5,FRYDMAN,PROF 6,NaN,NaN,NaN,x
6,GARBER,PROF 7,NaN,NaN,x,NaN
7,MARTINEZ,PROF 8,x,NaN,NaN,x
8,MASCIANICA,PROF 9,NaN,NaN,x,NaN
9,PETELIN,PROF 10,NaN,NaN,x,NaN
